<a href="https://colab.research.google.com/github/hongshi97/python-practice/blob/master/DM%20Project_BERT_ing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 분석 데이터 불러오기 및 전처리


In [ ]:
!pip install transformers

     |████████████████████████████████| 1.5MB 6.9MB/s 
     |████████████████████████████████| 890kB 20.0MB/s 
     |████████████████████████████████| 2.9MB 25.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=60ef09597c8736dd189ab54d939ee9d91511ec78512abede49a0791ca1cc8f10
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/bert_suicide2/khubert_korean_train_upsample.csv", index_col=0)
test = pd.read_csv("/content/drive/MyDrive/bert_suicide2/khubert_korean_test.csv")
train

,document,label
1,가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 건강하지 못한 편이다. ...,0
2,가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 보통이다. 남자다. 성관...,0
3,가정의 경제적 상태는 중하다. 평상시 자신의 건강상태는 보통이다. 남자다. 성관...,0
4,가정의 경제적 상태는 중상이다. 평상시 자신의 건강상태는 건강한 편이다. 남자다...,0
5,가정의 경제적 상태는 하다. 평상시 자신의 건강상태는 보통이다. 남자다. 성관계...,0
...,...,...
59222,가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 건강한 편이다. 여자다....,1
59223,가정의 경제적 상태는 중상이다. 평상시 자신의 건강상태는 보통이다. 여자다. 성...,1
59224,가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 매우 건강한 편이다. 여...,1
59225,가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 건강하지 못한 편이다. ...,1


In [ ]:
trainX = train.drop('label',axis=1) 
trainy = train['label']

testX = test.drop('label', axis=1)
testy = test['label']

In [ ]:
train = pd.concat([trainX,trainy], axis=1).reset_index(drop=True)
test = pd.concat([testX, testy], axis=1).reset_index(drop=True)

In [ ]:
print(train)
print('\n')
print(test)

                                                document  label
0        가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 건강하지 못한 편이다. ...      0
1        가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 보통이다. 남자다. 성관...      0
2        가정의 경제적 상태는 중하다. 평상시 자신의 건강상태는 보통이다. 남자다. 성관...      0
3        가정의 경제적 상태는 중상이다. 평상시 자신의 건강상태는 건강한 편이다. 남자다...      0
4        가정의 경제적 상태는 하다. 평상시 자신의 건강상태는 보통이다. 남자다. 성관계...      0
...                                                  ...    ...
59221    가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 건강한 편이다. 여자다....      1
59222    가정의 경제적 상태는 중상이다. 평상시 자신의 건강상태는 보통이다. 여자다. 성...      1
59223    가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 매우 건강한 편이다. 여...      1
59224    가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 건강하지 못한 편이다. ...      1
59225    가정의 경제적 상태는 중상이다. 평상시 자신의 건강상태는 건강한 편이다. 여자다...      1

[59226 rows x 2 columns]


       Unnamed: 0                                           document  label
0               3    가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 매우 건강한 편이다. 남...      0
1               8    가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 

In [ ]:
trainy.value_counts()

1    29613
0    29613
Name: label, dtype: int64

In [ ]:
testy.value_counts()

0    12737
1     1795
Name: label, dtype: int64

## 버트 인풋 만들기

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

버트를 사용하기에 앞서 가장 기초에 속하는 tokenizer 사용 방법에 대해서 잠시 배워보도록 하겠습니다.  
tokenizer.encode => 문장을 버트 모델의 인풋 토큰값으로 바꿔줌  
tokenizer.tokenize => 문장을 토큰화

In [ ]:
def convert_data(data_df):
    global tokenizer
    
    SEQ_LEN = 815 #SEQ_LEN : 버트에 들어갈 인풋의 길이
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode(data_df[document][i], max_length=SEQ_LEN, truncation=True, padding='max_length')
       
        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        # 정답(긍정 : 1 부정 0)을 targets 변수에 저장해 줌
        targets.append(data_df[label][i])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정    
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[document] = data_df[document].astype(str)
    data_df[label] = data_df[label].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

SEQ_LEN = 815
BATCH_SIZE = 20
# 설명변수 문장을 포함하고 있는 칼럼
document = "document"
# 자살 생각 여부 ( 0: 없음, 1: 있음 )를 포함하고 있는 칼럼
label = "label"

# train 데이터를 버트 인풋에 맞게 변환
train_x, train_y = load_data(train)

NameError: ignored

In [ ]:
test

,Unnamed: 0,document,label
0,3,가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 매우 건강한 편이다. 남...,0
1,8,가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 건강한 편이다. 남자다....,0
2,27,가정의 경제적 상태는 중하다. 평상시 자신의 건강상태는 건강한 편이다. 남자다....,0
3,29,가정의 경제적 상태는 상이다. 평상시 자신의 건강상태는 매우 건강한 편이다. 남...,0
4,31,가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 매우 건강한 편이다. 남...,0
...,...,...,...
14527,57291,가정의 경제적 상태는 하다. 평상시 자신의 건강상태는 매우 건강한 편이다. 여자...,0
14528,57292,가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 건강한 편이다. 여자다....,1
14529,57293,가정의 경제적 상태는 상이다. 평상시 자신의 건강상태는 건강한 편이다. 여자다....,0
14530,57296,가정의 경제적 상태는 중상이다. 평상시 자신의 건강상태는 매우 건강한 편이다. ...,0


In [ ]:
# 훈련 성능을 검증한 test 데이터를 버트 인풋에 맞게 변환
test_x, test_y = load_data(test)

100%|██████████| 14532/14532 [00:49<00:00, 293.22it/s]


## 버트를 활용한 감성분석 모델 만들기

버트 훈련을 빠르게 하기 위해, TPU를 사용하도록 하겠습니다.  
TPU를 사용하시고 싶지 않으신 분은 그냥 TPU 관련 부분을 실행하지 않으면 되겠습니다.

In [ ]:
# TPU 객체 지정
TPU = True
if TPU:
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
else:
  pass

INFO:absl:Entering into master device scope: /job:worker/replica:0/task:0/device:CPU:0


INFO:tensorflow:Initializing the TPU system: grpc://10.74.10.114:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.74.10.114:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [ ]:
model = TFBertModel.from_pretrained('bert-base-multilingual-cased')
# 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
# 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])
# 버트 아웃풋의 텐서의 shape은 [batch_size, 문장의 길이, 768]임

bert_outputs = bert_outputs[1]
sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(bert_outputs)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f60e7923e58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).WARNING:tensorflow:AutoGraph could not transform <bound method Socket.send of <zmq.sugar.socket.Socket object at 0x7f60d37dd660>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f60e7923e58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: <cyfunction Socket.send at 0x7f60e7923e58> is not a module, class, method, function, traceback, frame, or code object
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [ ]:
sentiment_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 815)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 815)]        0                                            
__________________________________________________________________________________________________
input_segment (InputLayer)      [(None, 815)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 177853440   input_word_ids[0][0]             
                                                                 input_masks[0][0]            

감정 분석에 맞는 버트 모형을 리턴하는 함수를 정의하도록 하겠습니다.  
TPU를 활용하려면 함수로 묶어야 합니다.

In [ ]:
# Rectified Adam 옵티마이저 사용
import tensorflow_addons as tfa
opt = tfa.optimizers.RectifiedAdam(lr=1.0e-5, weight_decay=0.0025)


sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['acc'])

In [ ]:
def create_sentiment_bert():
  # 버트 pretrained 모델 로드
  model = TFBertModel.from_pretrained('bert-base-multilingual-cased')
  # 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
  token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
  mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
  segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
  # 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
  bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

  bert_outputs = bert_outputs[1]
  sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(bert_outputs)
  sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)

  sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['acc'])
  return sentiment_model

## 훈련 및 성능 검증

In [ ]:
# TPU 실행 시
if TPU:
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
# 함수를 strategy.scope로 묶어 줌
  with strategy.scope():
    sentiment_model = create_sentiment_bert()
  
  sentiment_model.fit(train_x, train_y, epochs=4, shuffle=True, batch_size=75, validation_data=(test_x, test_y))
else:
  # GPU 모드로 훈련시킬 때
  sentiment_model = create_sentiment_bert()
  
  sentiment_model.fit(train_x, train_y, epochs=4, shuffle=True, batch_size=75, validation_data=(test_x, test_y))

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

Epoch 1/4


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


790/790 [==============================] - ETA: 0s - loss: 0.6161 - acc: 0.6378

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


790/790 [==============================] - 645s 699ms/step - loss: 0.6160 - acc: 0.6379 - val_loss: 0.4689 - val_acc: 0.7330
Epoch 2/4
790/790 [==============================] - 484s 613ms/step - loss: 0.4901 - acc: 0.7667 - val_loss: 0.5165 - val_acc: 0.7727
Epoch 3/4
790/790 [==============================] - 484s 613ms/step - loss: 0.4811 - acc: 0.7738 - val_loss: 0.5078 - val_acc: 0.7671
Epoch 4/4
790/790 [==============================] - 484s 613ms/step - loss: 0.4749 - acc: 0.7777 - val_loss: 0.4462 - val_acc: 0.7587


훈련한 모델을 path에 저장
path는 임의로 지정해 주세요

In [ ]:
# PATH는 임의로 지정

path = "/content/drive/MyDrive/BERT Data"

In [ ]:
sentiment_model.save_weights(path+"/21.01.06.SUI_BERT.h5")

Training된 가중치 들고와서 테스트할 때

In [ ]:
# Rectified Adam 옵티마이저 사용
import tensorflow_addons as tfa
opt = tfa.optimizers.RectifiedAdam(lr=1.0e-5, weight_decay=0.0025)


sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['acc'])

In [ ]:
sentiment_model = create_sentiment_bert()
sentiment_model.load_weights(path + "/21.01.06.SUI_BERT.h5")

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True

In [ ]:
sentiment_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 815)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 815)]        0                                            
__________________________________________________________________________________________________
input_segment (InputLayer)      [(None, 815)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_2 (TFBertModel)   TFBaseModelOutputWit 177853440   input_word_ids[0][0]             
                                                                 input_masks[0][0]          

훈련 모델의 예측 성능을 F1 SCORE로 체크하기 위한 작업

In [ ]:
DATA_COLUMN = 'document'

def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []
    
    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN][i], max_length=SEQ_LEN, truncation=True, padding='max_length')
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

In [ ]:
train_x[[document]]

,document
1,가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 건강하지 못한 편이다. ...
2,가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 보통이다. 남자다. 성관...
3,가정의 경제적 상태는 중하다. 평상시 자신의 건강상태는 보통이다. 남자다. 성관...
4,가정의 경제적 상태는 중상이다. 평상시 자신의 건강상태는 건강한 편이다. 남자다...
5,가정의 경제적 상태는 하다. 평상시 자신의 건강상태는 보통이다. 남자다. 성관계...
...,...
59222,가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 건강한 편이다. 여자다....
59223,가정의 경제적 상태는 중상이다. 평상시 자신의 건강상태는 보통이다. 여자다. 성...
59224,가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 매우 건강한 편이다. 여...
59225,가정의 경제적 상태는 중이다. 평상시 자신의 건강상태는 건강하지 못한 편이다. ...


In [ ]:
train_all = pd.read_csv("/content/drive/MyDrive/bert_suicide2/khubert_korean_train_upsample.csv", index_col=0)
train_x = train_all[['document']]
train_x.reset_index(drop=True, inplace=True)
train_set = predict_load_data(train_x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 59226/59226 [03:14<00:00, 303.75it/s]
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


CancelledError: ignored

In [ ]:
train_set

[array([[  101,  8843, 16605, ...,     0,     0,     0],
        [  101,  8843, 16605, ...,     0,     0,     0],
        [  101,  8843, 16605, ...,     0,     0,     0],
        ...,
        [  101,  8843, 16605, ...,     0,     0,     0],
        [  101,  8843, 16605, ...,     0,     0,     0],
        [  101,  8843, 16605, ...,     0,     0,     0]]),
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]]),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])]

In [ ]:
preds = sentiment_model.predict(train_set)
y_true = train_all['label']

CancelledError: ignored

test 데이터 예측하기 - 변수 10개 모두 사용

In [ ]:
# test_all = pd.read_csv('/content/drive/MyDrive/버트method총정리/test_set(mda)_korean.csv',index_col=0)
test_all = pd.read_csv("/content/drive/MyDrive/bert_suicide2/khubert_korean_test.csv", index_col = 0)
test_x= test_all[['document']]
# 훈련 성능을 검증한 test 데이터를 버트 인풋에 맞게 변환
test_x.reset_index(drop=True, inplace=True)
test_set = predict_load_data(test_x)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  0%|          | 0/14532 [00:00<?, ?it/s]


KeyError: ignored

In [ ]:
preds = sentiment_model.predict(test_set)

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


In [ ]:
y_true = test_all['label']

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

def model_evaluation(label, predict):
    cf_matrix = confusion_matrix(label, predict)
    Accuracy = (cf_matrix[0][0] + cf_matrix[1][1]) / sum(sum(cf_matrix))
    Precision = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[0][1])
    Recall = cf_matrix[1][1] / (cf_matrix[1][1] + cf_matrix[1][0])
    Specificity = cf_matrix[0][0] / (cf_matrix[0][0] + cf_matrix[0][1])
    F1_Score = (2 * Recall * Precision) / (Recall + Precision)
    Auc = roc_auc_score(y_true, np.round(preds,0))
    print("Accuracy: ", Accuracy)
    print("Precision: ", Precision)
    print("Recall: ", Recall)
    print("Specificity: ", Specificity)
    print("F1-Score: ", F1_Score)
    print("auc", roc_auc_score(y_true, np.round(preds,0)))

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

print("Confusion Matrix")
print(confusion_matrix(y_true, np.round(preds,0)))
print("="*32)
print("Model Evaluation")
print(model_evaluation(y_true, np.round(preds,0)))


Confusion Matrix
[[9570 3167]
 [ 397 1398]]
Model Evaluation
Accuracy:  0.7547481420313791
Precision:  0.30624315443592554
Recall:  0.7788300835654596
Specificity:  0.7513543220538589
F1-Score:  0.4396226415094339
auc 0.7650922028096592
None
